In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from IPython.display import display, Latex

In [2]:
df = pd.read_csv('../results/output.csv')
df['sampling'] = df['sampling'].replace({'normal_at_min': 'Normal', 'uniform': 'Uniform'})
df.head()

,func,noise,sampling,sample_sz,seed,learning,v_domain,modval_modsol,truval_modsol,funval_err,optsol_err,optval_err,opt_setup_time,opt_opt_time,ML_train_time,R2_score
0,beale,0.0,Normal,1000,2023,forest,box,2.152398,1.104498,1.047900,1.353093,2.152398,0.728047,0.392749,0.328566,0.938916
1,beale,0.0,Normal,1000,2023,forest,ch,2.152398,0.606251,1.546147,1.066877,2.152398,0.730197,0.794865,0.328566,0.938916
2,beale,0.0,Normal,1000,2023,forest,chplus,2.152398,1.002497,1.149901,1.313069,2.152398,0.740183,0.761531,0.328566,0.938916
3,beale,0.0,Normal,1000,2023,forest,isofor,2.152398,0.297833,1.854565,0.874175,2.152398,15.555727,1.414174,0.328566,0.938916
4,beale,0.0,Normal,1000,2023,forest,chp.05,2.152398,0.177445,1.974953,0.090352,2.152398,0.565199,0.325046,-1.000000,-1.000000


In [3]:
df_mean = df[~df['func'].isin(['rastrigin2d', 'rotated_hyper_ellipsoid'])]

df_mean = df_mean[~df_mean['v_domain'].isin(['chp.05', 'chp.1'])]

sampling_order = ['Uniform', 'Normal']
v_domain_order = ['box', 'ch', 'isofor', 'chplus', 'chp.05', 'chp.1']

df_mean = df_mean.assign(
    sampling=pd.Categorical(df_mean['sampling'], categories=sampling_order, ordered=True)
)

df_mean = df_mean.assign(
    v_domain=pd.Categorical(df_mean['v_domain'], categories=v_domain_order, ordered=True)
)

df_mean = df_mean.sort_values(['sampling', 'func', 'v_domain'])
df_mean_normal = (df_mean.groupby(['sampling','func', 'v_domain'], observed=True)[['funval_err', 'optsol_err', 'optval_err']].
 mean().
 reset_index().
 groupby(['sampling', 'func'], observed=True)[['v_domain','funval_err', 'optsol_err', 'optval_err']].
 apply(lambda g: g.iloc[:, 1:] / g.iloc[0, 1:]).
 reset_index(drop=True))

df_mean_normalized = (df_mean.groupby(['sampling','func', 'v_domain'], observed=True)[['funval_err', 'optsol_err', 'optval_err']].
 mean().
 reset_index())

df_mean_normalized[['funval_err', 'optsol_err', 'optval_err']] = df_mean_normal

df_mean_paper = (df_mean_normalized.
                 pivot(index=['func', 'v_domain'],
                       columns='sampling',
                       values=['funval_err','optsol_err', 'optval_err']).
                 map(lambda x: f'{x:.2f}'))

df_mean_paper

funval_err        optsol_err        optval_err       
sampling                 Uniform Normal    Uniform Normal    Uniform Normal
func         v_domain                                                      
beale        box            1.00   1.00       1.00   1.00       1.00   1.00
             ch             0.99   0.80       1.01   1.09       1.00   0.86
             isofor         0.73   0.50       0.78   0.49       0.88   0.64
             chplus         0.21   0.31       0.81   0.77       0.42   0.52
griewank     box            1.00   1.00       1.00   1.00       1.00   1.00
             ch             0.57   0.80       0.82   0.92       1.07   1.00
             isofor         0.63   0.76       0.30   0.54       1.68   1.00
             chplus         0.38   0.31       0.71   0.71       1.25   0.97
peaks        box            1.00   1.00       1.00   1.00       1.00   1.00
             ch             1.00   1.00       0.97   1.00       1.00   1.00
             isofor         1.21   2.13       0.92   0.50       1.10   1.00
             chplus         1.01   0.59       0.93   0.85       1.01   0.99
powell       box            1.00   1.00       1.00   1.00       1.00   1.00
             ch             0.99   0.86       0.95   0.88       0.99   0.91
             isofor         0.90   0.63       0.69   0.55       0.99   0.88
             chplus         0.50   0.29       0.79   0.58       0.63   0.66
qing         box            1.00   1.00       1.00   1.00       1.00   1.00
             ch             0.87   0.44       0.80   0.75       1.00   1.00
             isofor         1.23   0.49       0.66   0.58       1.00   1.00
             chplus         0.64   0.28       0.68   0.56       0.91   0.96
quintic      box            1.00   1.00       1.00   1.00       1.00   1.00
             ch             0.86   0.54       0.89   0.65       0.95   0.66
             isofor         0.33   0.23       0.69   0.48       0.95   0.59
             chplus         0.20   0.09       0.79   0.49       1.00   0.60
rastrigin10d box            1.00   1.00       1.00   1.00       1.00   1.00
             ch             0.79   0.37       0.84   0.53       1.02   1.10
             isofor         0.84   0.45       0.82   0.52       1.03   1.08
             chplus         0.69   0.33       0.71   0.50       1.09   1.18

In [4]:
display(Latex(df_mean_paper.to_latex(index=True, label="tab:mean")))

<IPython.core.display.Latex object>

In [5]:
text = df_mean_paper.to_latex(index=True, label="tab:mean")
with open('../results/tables/section_s2_errors_mean.tex', 'w') as f:
    f.write(text)